# Using TensorFlow to classify audio

This notebook aims to showcase an audio classification workflow using the UrbanSound8K dataset. The objective is to train a convolutional neural network (CNN) model to classify audio clips into two sound categories - "dog_bark" and "siren" which have been extracted from the UrbanSound8K dataset. The model takes Mel spectrogram representations of the audio data as input, which displays frequency information over time. These spectrograms are then fed into a CNN that is configured for audio analysis. The CNN learns to detect patterns that are predictive of each class. Once trained, the model can be used to classify new audio samples by extracting Mel spectrograms from the raw audio data and passing them through the CNN to obtain predicted probabilities for each class. 

This end-to-end workflow, from data preprocessing to model evaluation, demonstrates how deep learning techniques can be applied to audio analysis problems like sound recognition. The UrbanSound8K dataset provides realistic urban noise audio clips that can be used to train machine listening models for various applications such as smart assistants and audio surveillance systems.

###  Importing Libraries and Defining Parameters

Imports TensorFlow, Keras, Librosa and other libraries needed for the audio classification project. Also defines folder structure for the UrbanSound8K dataset, target audio classes ("dog_bark", "siren"), and model hyperparameters.

In [28]:
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.image import resize
from tensorflow.keras.callbacks import ModelCheckpoint
import warnings
warnings.filterwarnings('ignore', category=UserWarning)  # Ignore UserWarnings

In [36]:
# Folder structure
data_dir = 'data'
classes = ['dog_bark', 'siren']

### Loading and Preprocessing UrbanSound8K Data

Loads audio files from UrbanSound8K folders, extracts Mel spectrograms using Librosa, resizes them to consistent shape, and assembles into training and test sets with one-hot encoded labels.

In [37]:
# Load and preprocess audio data
def load_and_preprocess_data(data_dir, classes, target_shape=(128, 128)):
    data = []
    labels = []

    for i, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        for filename in os.listdir(class_dir):
            if filename.endswith('.wav'):
                file_path = os.path.join(class_dir, filename)
                audio_data, sample_rate = librosa.load(file_path, sr=None)
                # Perform preprocessing (convert to Mel spectrogram and resize)
                mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
                mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)
                data.append(mel_spectrogram)
                labels.append(i)

    return np.array(data), np.array(labels)

In [38]:
# Split data into training and testing sets
data, labels = load_and_preprocess_data(data_dir, classes)
labels = to_categorical(labels, num_classes=len(classes))  # Convert labels to one-hot encoding
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

### Building the Convolutional Neural Network

Defines a convolutional neural network architecture in TensorFlow/Keras with convolutional, pooling, dense layers suited for audio classification.

In [39]:
# Create a neural network model
input_shape = X_train[0].shape
input_layer = Input(shape=input_shape)
x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = BatchNormalization()(x)
x = Dropout(0.25)(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = BatchNormalization()(x)
x = Dropout(0.25)(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(len(classes), activation='softmax')(x)
model = Model(input_layer, output_layer)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 128, 128, 1)]     0         
                                                                 
 conv2d_4 (Conv2D)           (None, 126, 126, 32)      320       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 63, 63, 32)        0         
 g2D)                                                            
                                                                 
 batch_normalization_4 (Bat  (None, 63, 63, 32)        128       
 chNormalization)                                                
                                                                 
 dropout_6 (Dropout)         (None, 63, 63, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 61, 61, 64)        1849

### Compiling the Model

Compiles CNN model with Adam optimizer and categorical cross-entropy loss function appropriate for multi-class prediction.

In [40]:
# Compile the model
model.compile(tf.keras.optimizers.legacy.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [41]:
# Define model checkpointing
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

### Training the Model on UrbanSound8K Data

Fits the model on UrbanSound8K training data for 200 epochs, validating on the test set after each epoch and saving the best model.

In [42]:
# Train the model
history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_test, y_test),
                    callbacks=[model_checkpoint])

Epoch 1/200
49/49 [==============================] - 5s 99ms/step - loss: 2.4643 - accuracy: 0.6546 - val_loss: 2.8031 - val_accuracy: 0.7513
Epoch 2/200
49/49 [==============================] - 5s 101ms/step - loss: 0.8681 - accuracy: 0.6811 - val_loss: 0.7967 - val_accuracy: 0.8031
Epoch 3/200
49/49 [==============================] - 5s 99ms/step - loss: 0.7586 - accuracy: 0.6954 - val_loss: 0.5831 - val_accuracy: 0.8238
Epoch 4/200
49/49 [==============================] - 5s 101ms/step - loss: 0.5014 - accuracy: 0.7200 - val_loss: 0.5658 - val_accuracy: 0.7565
Epoch 5/200
49/49 [==============================] - 5s 100ms/step - loss: 0.4864 - accuracy: 0.7485 - val_loss: 0.5317 - val_accuracy: 0.7435
Epoch 6/200
49/49 [==============================] - 5s 100ms/step - loss: 0.4637 - accuracy: 0.7317 - val_loss: 0.4459 - val_accuracy: 0.7668
Epoch 7/200
49/49 [==============================] - 5s 107ms/step - loss: 0.4335 - accuracy: 0.7706 - val_loss: 0.4213 - val_accuracy: 0.8446
E

49/49 [==============================] - 5s 95ms/step - loss: 0.2095 - accuracy: 0.8820 - val_loss: 0.5877 - val_accuracy: 0.9067
Epoch 116/200
49/49 [==============================] - 5s 99ms/step - loss: 0.2203 - accuracy: 0.8911 - val_loss: 0.4912 - val_accuracy: 0.8705
Epoch 117/200
49/49 [==============================] - 5s 97ms/step - loss: 0.1939 - accuracy: 0.8995 - val_loss: 0.7080 - val_accuracy: 0.9197
Epoch 118/200
49/49 [==============================] - 5s 97ms/step - loss: 0.1963 - accuracy: 0.8937 - val_loss: 0.8163 - val_accuracy: 0.8627
Epoch 119/200
49/49 [==============================] - 5s 95ms/step - loss: 0.1965 - accuracy: 0.8957 - val_loss: 0.8680 - val_accuracy: 0.8627
Epoch 120/200
49/49 [==============================] - 5s 92ms/step - loss: 0.1888 - accuracy: 0.8957 - val_loss: 0.8481 - val_accuracy: 0.8446
Epoch 121/200
49/49 [==============================] - 5s 93ms/step - loss: 0.1775 - accuracy: 0.8924 - val_loss: 1.1199 - val_accuracy: 0.8549
Epoch 

Epoch 172/200
49/49 [==============================] - 5s 93ms/step - loss: 0.1539 - accuracy: 0.9222 - val_loss: 0.8069 - val_accuracy: 0.8342
Epoch 173/200
49/49 [==============================] - 5s 93ms/step - loss: 0.1547 - accuracy: 0.9190 - val_loss: 0.6427 - val_accuracy: 0.8653
Epoch 174/200
49/49 [==============================] - 5s 94ms/step - loss: 0.1596 - accuracy: 0.9216 - val_loss: 0.2323 - val_accuracy: 0.9067
Epoch 175/200
49/49 [==============================] - 5s 93ms/step - loss: 0.1550 - accuracy: 0.9255 - val_loss: 0.6037 - val_accuracy: 0.8368
Epoch 176/200
49/49 [==============================] - 5s 100ms/step - loss: 0.1461 - accuracy: 0.9268 - val_loss: 0.3349 - val_accuracy: 0.8912
Epoch 177/200
49/49 [==============================] - 5s 96ms/step - loss: 0.1457 - accuracy: 0.9274 - val_loss: 0.5734 - val_accuracy: 0.8705
Epoch 178/200
49/49 [==============================] - 5s 98ms/step - loss: 0.1446 - accuracy: 0.9281 - val_loss: 0.3893 - val_accuracy

### Loading and Evaluating the Best Model

Reloads the best saved model and evaluates its test accuracy on UrbanSound8K dataset.

In [43]:
# Load the best saved model
best_model = tf.keras.models.load_model('best_model.h5')

### Testing Model on Sample Audio

In this stage we pass an audio file through the trained model to predict class and then we print the prediction accuracy.

In [44]:
# Test the best model
test_accuracy = best_model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy:", test_accuracy[1])

Test Accuracy: 0.9067357778549194


In [45]:
# Define the target shape for input spectrograms
target_shape = (128, 128)

In [46]:

# Function to preprocess and classify an audio file
def test_audio(file_path, model):
    # Load and preprocess the audio file
    audio_data, sample_rate = librosa.load(file_path, sr=None)
    mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
    mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)
    mel_spectrogram = tf.reshape(mel_spectrogram, (1,) + target_shape + (1,))

    # Make predictions
    predictions = model.predict(mel_spectrogram)

    # Get the class probabilities
    class_probabilities = predictions[0]

    # Get the predicted class index
    predicted_class_index = np.argmax(class_probabilities)

    return class_probabilities, predicted_class_index

In [50]:
# Test an audio file
test_audio_file = 'dog01.wav'
class_probabilities, predicted_class_index = test_audio(test_audio_file, best_model)

1/1 [==============================] - 0s 44ms/step


In [51]:
# Display results for all classes
for i, class_label in enumerate(classes):
    probability = class_probabilities[i]
    print(f'Class: {class_label}, Probability: {probability:.4f}')

Class: dog_bark, Probability: 1.0000
Class: siren, Probability: 0.0000


In [52]:
# Calculate and display the predicted class and accuracy
predicted_class = classes[predicted_class_index]
accuracy = class_probabilities[predicted_class_index]
print(f'The audio is classified as: {predicted_class}')
print(f'Accuracy: {accuracy:.4f}')

The audio is classified as: dog_bark
Accuracy: 1.0000
